<a href="https://colab.research.google.com/github/simoncuet/Assistive-Application-for-Visually-Impaired-Children/blob/master/Bengali_AI_Handwritten_Grapheme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
%tensorflow_version 1.x
import numpy as np
import pandas as pd
import cv2
import gc

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
from tensorflow import keras
from keras import applications
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.utils import plot_model
from keras.models import Model, load_model
from tensorflow.keras import layers
from keras.initializers import glorot_uniform
from sklearn.model_selection import train_test_split

# Connect to my google drive where train and test data are located
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
num_purquet = 4 # both for test and train images
img_w, img_h = 137, 236
resize_w, resize_h = 64, 64

# train data
train_csv_path = '/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train.csv'
train_img_path = []
for i in range(num_purquet):
  train_img_path.append(f'/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_{i}.parquet')

# test data
test_csv_path = '/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/test.csv'
test_img_path = []
for i in range(num_purquet):
  test_img_path.append(f'/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/test_image_data_{i}.parquet')

In [3]:
train_label = pd.read_csv(train_csv_path)
train_data_list = []
y_root = []
y_vowel = []
y_consonant = []
for file_path in train_img_path:
  resized = {}
  df =  pd.merge(pd.read_parquet(file_path), train_label, on='image_id').drop(['image_id'], axis =1)
  print(f'data loaded from {file_path}')
  y_root.append(df['grapheme_root'].values)
  y_vowel.append(df['vowel_diacritic'].values)
  y_consonant.append(df['consonant_diacritic'].values)
  df = df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme'], axis=1)
  for i in range(df.shape[0]):
    if i%10000==0:
      print(i)
    img = cv2.resize(df.loc[df.index[i]].values.reshape(img_w,img_h),(resize_w,resize_h))
    #img=img/255
    resized[df.index[i]] = img.reshape(-1)
  resized = pd.DataFrame(resized).T
  train_data_list.append(resized)
  del df
  gc.collect()
x = pd.concat(train_data_list, ignore_index=True)
del train_data_list
gc.collect()

data loaded from /content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_0.parquet
0
10000
20000
30000
40000
50000
data loaded from /content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_1.parquet
0
10000
20000
30000
40000
50000
data loaded from /content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_2.parquet
0
10000
20000
30000
40000
50000
data loaded from /content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_3.parquet
0
10000
20000
30000
40000
50000


0

In [0]:
y_root = np.concatenate(y_root, axis = None)
y_vowel = np.concatenate(y_vowel, axis = None)
y_consonant = np.concatenate(y_consonant, axis = None)

In [0]:
x_train, x_test, y_root_train, y_root_test, y_vowel_train, y_vowel_test, y_consonant_train, y_consonant_test = train_test_split(x, y_root, y_vowel, y_consonant, test_size = 0.08, random_state=42)

In [6]:
print(x_train.shape, x_test.shape)
print(y_root_train.shape, y_root_test.shape)
print(y_vowel_train.shape, y_vowel_test.shape)
print(y_consonant_train.shape, y_consonant_test.shape)

(184772, 4096) (16068, 4096)
(184772,) (16068,)
(184772,) (16068,)
(184772,) (16068,)


In [7]:
del x
del y_root
del y_vowel
del y_consonant
gc.collect()

0

In [0]:

def identity_block(X, f, filters, stage, block):
    
    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

def convolutional_block(X, f, filters, stage, block, s=2):

    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    
    # Define the input as a tensor with shape input_shape
    X_input =Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2,2), padding='same')(X)

    # Output layer
    X = Flatten()(X)
    model = Dense(1024, activation = "relu")(X)
    #model = layers.Dropout(rate=0.3)(model)
    dense = Dense(512, activation = "relu")(model)
    #X = Dense(classes[0], activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    out_root = Dense(classes[0], activation='softmax', name='root_out') (dense)
    out_vowel = Dense(classes[1], activation='softmax', name='vowel_out') (dense)
    out_consonant = Dense(classes[2], activation='softmax', name='consonant_out')(dense)

    model = Model(inputs = X_input, outputs = [out_root, out_vowel, out_consonant])
    model.compile(optimizer = tf.train.AdamOptimizer(), loss=['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'], metrics=['accuracy'])
  
    
    
    # Create model
    #model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model


In [0]:
model = ResNet50(input_shape = (64, 64, 1), classes = [168,11,7])
history = model.fit(x_train.values.reshape(-1, resize_w, resize_h,1), {'root_out':y_root_train, 'vowel_out':y_vowel_train, 'consonant_out':y_consonant_train}, epochs=20, batch_size = 128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
 61184/184772 [========>.....................] - ETA: 2:28:22 - loss: 7.3446 - root_out_loss: 4.6079 - vowel_out_loss: 1.6159 - consonant_out_loss: 1.1209 - root_out_acc: 0.0404 - vowel_out_acc: 0.4438 - consonant_out_acc: 0.6360

In [0]:
def get_model():
  inputs = keras.Input(shape=(resize_w, resize_h,1))
  #CONV 1A STARTS
  model = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(inputs)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.BatchNormalization(momentum=0.15)(model)
  model = layers.MaxPool2D(pool_size=(2, 2))(model)
  model = layers.Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.Dropout(rate=0.3)(model)
  #CONV 1B STARTS
  model = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(inputs)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.BatchNormalization(momentum=0.15)(model)
  model = layers.MaxPool2D(pool_size=(2, 2))(model)
  model = layers.Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.Dropout(rate=0.3)(model)
  #CONV 2A STARTS
  model = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.BatchNormalization(momentum=0.15)(model)
  model = layers.Dropout(rate=0.3)(model)
  #CONV 2B STARTS
  model = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.BatchNormalization(momentum=0.15)(model)
  model = layers.Dropout(rate=0.3)(model)
  #CONV 3A STARTS
  model = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.BatchNormalization(momentum=0.15)(model)
  model = layers.MaxPool2D(pool_size=(2, 2))(model)
  #CONV 3B STARTS
  model = layers.Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
  model = tf.nn.leaky_relu(model, alpha=0.01, name='Leaky_ReLU') 
  model = layers.BatchNormalization(momentum=0.15)(model)
  model = layers.Dropout(rate=0.3)(model)
  
  model = layers.Flatten()(model)
  model = layers.Dense(1024, activation = "relu")(model)
  model = layers.Dropout(rate=0.3)(model)
  dense = layers.Dense(512, activation = "relu")(model)

  #Decision
  out_root = layers.Dense(168, activation='softmax', name='root_out') (dense)
  out_vowel = layers.Dense(11, activation='softmax', name='vowel_out') (dense)
  out_consonant = layers.Dense(7, activation='softmax', name='consonant_out')(dense)

  model = keras.Model(inputs = inputs, outputs = [out_root, out_vowel, out_consonant])
  model.compile(optimizer = tf.train.AdamOptimizer(), loss=['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'], metrics=['accuracy'])
  
  return model

In [13]:
model = get_model()
history = model.fit(x_train.values.reshape(-1, resize_w, resize_h,1), {'root_out':y_root_train, 'vowel_out':y_vowel_train, 'consonant_out':y_consonant_train}, epochs=20, batch_size = 256)

Train on 184772 samples
Epoch 1/20
157696/184772 [========================>.....] - ETA: 14:00 - loss: 5.1192 - root_out_loss: 3.5922 - vowel_out_loss: 0.8026 - consonant_out_loss: 0.7244 - root_out_acc: 0.1645 - vowel_out_acc: 0.7238 - consonant_out_acc: 0.7535

KeyboardInterrupt: ignored

In [10]:
model.evaluate(x_test.values.reshape(-1, 64, 64, 1), {'root_out':y_root_test, 'vowel_out':y_vowel_test, 'consonant_out':y_consonant_test})

#print('Test accuracy:', test_acc)

4017/4017 [==============================] - 7s 2ms/sample - loss: 2.1465 - root_out_loss: 1.5527 - vowel_out_loss: 0.2995 - consonant_out_loss: 0.2985 - root_out_acc: 0.5676 - vowel_out_acc: 0.8992 - consonant_out_acc: 0.9014


[2.1465109240987985,
 1.5526849,
 0.29947168,
 0.29846987,
 0.56758773,
 0.8991785,
 0.901419]

In [0]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet('../input/bengaliai-cv19/test_image_data_{}.parquet'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)
    X_test = resize(df_test_img, need_progress_bar=False)/255
    X_test = X_test.values.reshape(-1, 64, 64, 1)
    preds = cnn1model.predict(X_test)
    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test
    gc.collect()
df_sample = pd.DataFrame({'row_id': row_id, 'target':target}, columns = ['row_id','target'])
df_sample.to_csv('submission.csv',index=False)
df_sample.head()

NameError: ignored

In [0]:
# Read train data into: df
df1 = pd.read_parquet('/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_0.parquet')
df2 = pd.read_parquet('/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_1.parquet')
df3 = pd.read_parquet('/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_2.parquet')
df4 = pd.read_parquet('/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_3.parquet')
list_df = [df1, df2, df3, df4]
df = pd.concat(list_df, ignore_index=True)


In [0]:
data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
df = pd.DataFrame(data)
df

NameError: ignored

In [0]:
train_data = pd.read_csv('/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train.csv')
data_list = []
for i in range(4):
  resized = {}
  print('data loaded..',i)
  df =  pd.merge(pd.read_parquet(f'/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train_image_data_{i}.parquet'), train_data, on='image_id').drop(['image_id'], axis =1)
  df = df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme'], axis=1)
  for i in range(df.shape[0]):
    if i%10000==0:
      print(i)
    #img = cv2.resize(df.loc[df.index[i]].values.reshape(137,236), (64,64))
    img = df.loc[df.index[i]].values.reshape(137,236)
    resized[df.index[i]] = img.reshape(-1)
  resized = pd.DataFrame(resized).T
  data_list.append(resized)
df = pd.concat(data_list, ignore_index=True)

data loaded.. 0
0
10000
20000
30000
40000
50000
data loaded.. 1
0
10000
20000
30000
40000
50000
data loaded.. 2
0
10000
20000
30000
40000
50000
data loaded.. 3
0
10000
20000
30000
40000
50000


In [0]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,32292,32293,32294,32295,32296,32297,32298,32299,32300,32301,32302,32303,32304,32305,32306,32307,32308,32309,32310,32311,32312,32313,32314,32315,32316,32317,32318,32319,32320,32321,32322,32323,32324,32325,32326,32327,32328,32329,32330,32331
0,254,253,252,253,251,252,253,251,251,253,254,253,253,253,254,253,252,253,253,253,253,252,252,253,253,252,252,253,252,252,252,253,254,253,253,252,252,252,253,252,...,252,252,252,252,252,252,252,252,252,252,252,252,252,252,252,252,252,252,252,252,252,252,253,253,253,253,253,253,253,253,253,253,253,253,253,253,253,253,253,251
1,251,244,238,245,248,246,246,247,251,252,250,250,246,249,248,250,249,251,252,253,253,253,253,253,253,253,250,249,251,252,251,251,251,251,252,253,251,250,252,251,...,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,254
2,251,250,249,250,249,245,247,252,252,252,253,252,252,251,250,251,253,254,251,251,252,252,253,253,252,251,251,249,251,252,252,253,252,251,251,251,250,250,252,253,...,253,253,252,252,252,253,253,253,253,253,252,251,251,250,250,250,251,251,251,250,250,250,251,252,253,253,253,253,254,254,254,253,252,252,253,253,253,253,251,249
3,247,247,249,253,253,252,251,251,250,250,251,250,249,251,251,251,250,252,251,245,245,251,252,251,252,252,250,249,250,251,250,249,250,251,252,253,252,252,252,252,...,253,252,252,254,253,253,254,253,252,253,254,253,252,253,254,254,254,254,254,254,254,254,253,252,253,254,253,252,253,254,254,254,254,254,254,253,253,252,251,252
4,249,248,246,246,248,244,242,242,229,225,231,229,229,228,221,224,226,221,221,220,217,217,218,219,222,224,214,218,227,227,227,228,224,231,235,235,233,212,183,196,...,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255


In [0]:
img_df = df.loc[3]
img_arr = img_df.values
img_arr = img_arr.reshape(137,236,1)
img_arr.shape
cv2.imwrite("hi.jpg", img_arr)

img = cv2.imread('hi.jpg', cv2.IMREAD_GRAYSCALE)
#img = cv2.resize(img, (225,225))
plt.imshow(img, cmap='gray')
plt.show()

NameError: ignored

In [0]:
 x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split
    ((
        resize(b_train_data.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme'], axis=1))/255).values.reshape(-1, 64, 64, 1), pd.get_dummies(
        b_train_data['grapheme_root']).values, pd.get_dummies(b_train_data['vowel_diacritic']).values, pd.get_dummies(b_train_data['consonant_diacritic']).values,

In [0]:
b_train_data.shape

(50210, 32336)

In [0]:
train_images = cv2.resize(df.iloc[:,1:].to_numpy().astype(int).reshape(137,236), (64,64))
train_images.shape
del df
gc.collect()

In [0]:

    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split((resize(
        b_train_data.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme'], axis=1))/255).values.reshape(-1, 64, 64, 1), pd.get_dummies(
        b_train_data['grapheme_root']).values, pd.get_dummies(b_train_data['vowel_diacritic']).values, pd.get_dummies(b_train_data['consonant_diacritic']).values,

In [0]:
list_img_arr = []
img_ids =[]
for i in range(df.shape[0]):
  if i%1000==0:
    print(i)
  img_arr = df.iloc[i,1:].to_numpy().astype(int).reshape(137,236)
  #train_images = np.append(train_images, img_arr)
  img_ids.append(df.iloc[i,0])
  list_img_arr.append(img_arr)
train_images = np.array(list_img_arr)
print(train_images.shape)
print(len(img_ids))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


KeyboardInterrupt: ignored

In [0]:
# Visualize handwritten image
img_df = df.iloc[0:1,1:]
img_arr = img_df.values
img_arr = img_arr.reshape(137,236,1)
img_arr.shape
cv2.imwrite("hi.jpg", img_arr)

img = cv2.imread('hi.jpg', cv2.IMREAD_GRAYSCALE)
#img = cv2.resize(img, (225,225))
plt.imshow(img, cmap='gray')
plt.show()

In [0]:
# Read train.csv
df_train = pd.read_csv('/content/drive/My Drive/content/Bengali.AI Handwritten Grapheme Classification/Dataset/Dataset/train.csv')

In [0]:
df_train.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [0]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
train_labels[0:5]

array([5, 0, 4, 1, 9], dtype=uint8)